In [1]:
# # Import comet_ml at the top of your file
# from comet_ml import Experiment

# # Create an experiment with your api key
# experiment = Experiment(
#     api_key="7IzkZqpQDneBnYdnl3QJMYV4K",
#     project_name="test-comet-project-name",
#     workspace="milosz-l",
# )

# # Report multiple hyperparameters using a dictionary:
# hyper_params = {
#     "learning_rate": 0.5,
#     "steps": 100000,
#     "batch_size": 50,
# }
# experiment.log_parameters(hyper_params)

# # Or report single hyperparameters:
# hidden_layer_size = 50
# experiment.log_parameter("hidden_layer_size", hidden_layer_size)

# # Long any time-series metrics:
# train_accuracy = 3.14
# experiment.log_metric("accuracy", train_accuracy, step=0)

# # Run your code and go to /

In [2]:
import comet_ml
from datasets import load_dataset

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, DataCollatorWithPadding

# comet_ml.init(project_name = "Hugging Face Text Classification")

comet_ml is installed but `COMET_API_KEY` is not set.


In [3]:
df = load_dataset("imdb")
print(df)

Found cached dataset imdb (/Users/milosz/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [8]:
set(df['train']['label'])

{0, 1}

In [5]:
PRE_TRAINED_MODEL_NAME = "distilbert-base-uncased"
random_seed = 42

In [6]:
# Import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
# experiment = Experiment(
#     api_key="7IzkZqpQDneBnYdnl3QJMYV4K",
#     project_name="hugging-face-text-classification",
#     workspace="milosz-l",
# )

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/milosz-l/hugging-face-text-classification/b8070fa34b81493ea3c203d1148fdda6



In [7]:
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

def tokenize_function(data):
    return tokenizer(data["text"], padding="max_length", truncation=True)


tokenize_df = df.map(tokenize_function, batched=True)
train_df = tokenize_df["train"].shuffle(seed=random_seed).select(range(200))
test_df = tokenize_df["test"].shuffle(seed=random_seed).select(range(100))

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, num_labels=2
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [9]:
training_arguments = TrainingArguments(
    seed=random_seed,
    optim="adamw_torch",
    learning_rate=5e-5,
    num_train_epochs=1,
    output_dir="./results",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_total_limit=10,
    save_steps=25
)

In [11]:
def compute_metrics(pred):
    
    #get global experiments
    # experiment = comet_ml.get_global_experiment()
    experiment = Experiment(
    api_key="7IzkZqpQDneBnYdnl3QJMYV4K",
    project_name="hugging-face-text-classification",
    workspace="milosz-l",
    )
    
    #get y_true and y_preds for eval_dataset
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    #compute precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='macro')
    
    #compute accuracy score
    acc = accuracy_score(labels, preds)
    
    #log confusion matrix
    if experiment:
        epoch = int(experiment.curr_epoch) if experiment.curr_epoch is not None else 0
        experiment.set_epoch(epoch)
        experiment.log_confusion_matrix(
            y_true=labels,
            y_predicted=preds,
            labels=["negative", "postive"]
        )

    return {"accuracy": acc, 
            "f1": f1, 
            "precision": precision,
            "recall": recall
            }

In [13]:
%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_df,
    eval_dataset=test_df,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)
trainer.train()

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 200
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25
  Number of trainable parameters = 66955010
COMET ERROR: Failed to extract scalar from SummaryWriter.add_hparams()


  0%|          | 0/25 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

...

Model weights saved in ./results/checkpoint-25/pytorch_model.bin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)




Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 105.1166, 'train_samples_per_second': 1.903, 'train_steps_per_second': 0.238, 'train_loss': 0.6340503692626953, 'epoch': 1.0}


TrainOutput(global_step=25, training_loss=0.6340503692626953, metrics={'train_runtime': 105.1166, 'train_samples_per_second': 1.903, 'train_steps_per_second': 0.238, 'train_loss': 0.6340503692626953, 'epoch': 1.0})